In [1]:
import torch
torch.set_default_tensor_type(torch.DoubleTensor)

import argparse
from   config            import *
from   UKDALE_Parser     import *
from   REDD_Parser       import *
from   Refit_Parser      import *
from PChainger_Parser import PChainger_Parser
from   Electricity_model import *
from   NILM_Dataloader   import *
from   Trainer           import *
from   time              import time
import pickle            as pkl
from pathlib import Path
import plotly.express as px
import plotly.io as pio

pio.renderers.default = 'iframe' # or 'colab' or 'iframe' or 'iframe_connected' or 'sphinx_gallery'

In [2]:
with open("./results_UK-DALE_TitanV_kettle/uk_dale/kettle/results.pkl", "rb") as f:
    res = pkl.load(f)

args = res["args"]

In [3]:
# override computing device
args.device = "cpu"
args.pretrain_num_epochs = 0
args.num_epochs = 0
args.validation_size = 1
args.hidden = 256

In [4]:
data_file = "./data/test/geert/main.csv"

ds_parser = PChainger_Parser(
    sampling="6s",
    normalize="mean",
    cutoff=[3100],
    threshold=[2000],
    location=data_file,
    min_on=[2],
    min_off=[0],
    val_size=1,
    window_size=480,
    window_stride=240
)

In [5]:
model = ELECTRICITY(args)
args.pretrain = False  # remove?
model.pretrain = False  # remove?
model.load_state_dict(torch.load("./results_UK-DALE_TitanV_kettle/uk_dale/kettle/best_acc_model.pth", map_location=torch.device('cpu')))
trainer = Trainer(args,ds_parser,model)

In [47]:
inf_data_start = 0
inf_data_end = 480
inference_data = NILMDataset(
    #x=ds_uk_parser.x[-100000:],
    x=ds_parser.x,#[-100000:],
    #y=ds_uk_parser.y[-100000:],
    y=[0] * len(ds_parser.x),#100000,
    #status=ds_uk_parser.status[-100000:],
    status=[0] * len(ds_parser.x),#100000,
    window_size=480,
    stride=480
)

In [48]:
inference_dataloader = torch.utils.data.DataLoader(
    dataset=inference_data,
    batch_size=inf_data_end-inf_data_start,
)

In [49]:
out = model.eval()

In [50]:
batches = []
for batch in inference_dataloader:
    batches.append(model(batch[0].to("cpu")))

data = []
for batch in batches:
    for chunk in batch[0]:
        data += list(chunk[0].tolist())

In [52]:
print(len(data))
print(len(ds_parser.index))
print(len(ds_parser.x))

132000
131976
131976


In [55]:
df_inf = pd.DataFrame(dict(
    #index = range(len(data)),
    #inf = data
    index = range(len(data)),#,(100000),
    inf = data#,[-100000:]
))

fig = px.line(df_inf, x="index", y="inf", line_shape="linear", render_mode="svg")
fig.show()

In [59]:
kettle = pd.read_csv("./data/test/geert/kettle.csv", sep=',', header=None, names=["time", "aggregate"])
kettle["time"] = pd.to_datetime(kettle["time"])
kettle = kettle.set_index("time")

In [60]:
df_kettle = pd.DataFrame(dict(
    index = kettle.index,#[-100000:],
    data = kettle["aggregate"],#[-100000:]
))

fig = px.line(df_kettle, x="index", y="data", line_shape="linear", render_mode="svg")
fig.show()

In [61]:
main = pd.read_csv("./data/test/geert/main.csv", sep=',', header=None, names=["time", "aggregate"])
main["time"] = pd.to_datetime(main["time"])
main = main.set_index("time")

In [62]:
df_main = pd.DataFrame(dict(
    index = main.index,#[-100000:],
    data = main["aggregate"],#[-100000:]
))

fig = px.line(df_main, x="index", y="data", line_shape="linear", render_mode="svg")
fig.show()